
## Dog Puppulation in Zürich: A Geospatial Neighborhood Analysis

### Introduction

#### Problem Statement:
Can we develop a data-driven model by the end of January 2024 that predicts what the dog *puppulation* density will be this year across Zürich’s 34 neighborhoods, with a Mean Absolute Error of less than 10%, using time series cross validation, to provide valuable insights for urban planning, pet-related businesses, and community welfare?


#### Context:
Following the City Council Resolution to override the Law on the Keeping of Dogs, the City of Zürich has embarked on a comprehensive exploration of dog *puppulation* dynamics in its neighborhoods. This initiative, prompted by that regulatory shift, aims to sniff out patterns in dog *puppulation* density that impact urban planning, business opportunities, and the overall welfare of our furry companions and their owners. The study leverages data from **2015** to **2020** to improve urban planning, boost pet-related business ventures, and foster community welfare through a better understanding of dog *puppulation* density patterns. This study is vital in this new era for Zürich, providing practical recommendations for the near future. The aim is to develop a data-driven model that reliably predicts the dog *puppulation* density across Zürich’s 34 neighborhoods in the near future.


#### Criteria for Success:
Our goal is to *dig up* clear patterns of dog *puppulation* density in Zürich’s neighborhoods, laying the groundwork for informed future predictions. We aim to *unleash* the potential of our predictive models, forecasting 2024 dog *puppulation* density patterns in Zürich with a Mean Absolute Error of less than 10%. Achieving this would be a *pawsitive* step towards informed future urban strategies.


#### Constraints within Solution Space:
- **Temporal Scope**: The study is confined to the years with full data availability across all datasets (2015-2020)
- **Spatial Resolution**: The study focuses on dog *puppulation* density at the neighborhood level. This may not capture variations within neighborhoods or between smaller areas.
- **Generalizability**: The findings of this study are specific to Zürich and may not be applicable to other cities or regions with different demographic, economic, and cultural contexts.


#### Stakeholders:
- **City Planners and Local Authorities:** Empower data-driven decision-making to enhance urban living conditions.
- **Business Enterprises:** Guide service offerings and marketing strategies.
- **Dog Owners:** Offer insights into community resources and pet care options.


#### Key Data Sources:
- **Geospatial Boundaries:** [Zürich Statistical Quarters](https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere)
- **Dog Ownership Records:** [Dog Owners Dataset](https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv)
- **Demographic Statistics:** [Population Dataset](https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv)
- **Economic Indicators:** [Income Dataset](https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv)
- **Household Dynamics:** [Household Size Dataset](https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv)

#### Analytical Objectives:
- **Understand the Relationship**: Dig into the relationship between demographic factors and dog *puppulation* density across Zürich’s neighborhoods.
- **Identify Trends and Clusters**: Track and map out the spatial and temporal trends of dog *puppulation* density. Identify spatial clusters of high and low dog *puppulation* density.
- **Predict Future Trends**: Predict the near-future trends of dog *puppulation* density using historical data, aiming for a Mean Absolute Error of less than 10%. This includes forecasting where Zürich’s dog *puppulation* will be booming across its 34 neighborhoods in the immediate future.


### Imports & Configurations

This section includes the necessary imports for libraries, configuration settings for dataframes and visualizations. These components establish the foundational setup for subsequent data analysis and exploration. 


In [ ]:
# Standard libraries
from functools import partial

from IPython.display import clear_output

import math

from PIL import ImageDraw, Image  # For image processing

from urllib.request import urlopen


# Related third party imports

from bokeh.models import FixedTicker, NumeralTickFormatter

import cartopy.crs as ccrs  # For cartographic projections and geographic plots

import colorcet as cc  # Additional color palettes

from esda.moran import Moran, Moran_Local  # Spatial autocorrelation statistics

from fiona.io import ZipMemoryFile

import geopandas as gpd

import geoviews as gv

import holoviews as hv

from holoviews import streams

import hvplot.pandas  # noqa

from matplotlib import pyplot as plt

import libpysal as lps  # Spatial analysis library

import numpy as np

import pandas as pd

import panel as pn

import panel.widgets as pnw
from pmdarima import auto_arima  # For determining ARIMA orders

import seaborn as sns

from splot.esda import plot_local_autocorrelation
from sklearn import metrics  # For evaluating model performance

from thefuzz import fuzz  # For string matching
from tqdm.notebook import tqdm  # Progress bars

from wordcloud import WordCloud  # For generating word cloud visualizations


# Local application/library specific imports

import helper_functions as hf  # Custom helper functions for this project

from translate_app import translate_list_to_dict


clear_output()

In [ ]:
# Additional configurations for visualization libraries
gv.extension("bokeh")
hv.extension("bokeh")
hvplot.extension("bokeh")
pn.extension(template="fast", nthreads=4, sizing_mode="stretch_width")
clear_output()

In [ ]:
# Pandas display options
# Disable warnings for chained assignments
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

# Seaborn style setting
sns.set_style("whitegrid")

# Panel configuration for improved interactivity performance
pn.config.throttled = True

# Clear any output created by the extensions and settings
clear_output()

### Data Description

This project utilizes various datasets to reveal the relationship between dog owner geodemographic factors and dog population density in Zurich. 




<table>
    <thead>
        <tr>
            <th>Dataset</th>
            <th>Source URL</th>
            <th>Original Source</th>
            <th>Description</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><a href="#Zurich-Statistical-Districts-Geospatial-Data">Zurich Districts Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere">Stadt Zürich</a></td>
            <td>Statistical Quarters</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Dogs-Dataset">Zurich Dogs Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001">Stadt Zürich</a></td>
            <td>Dog populations of the City of Zurich since 2015.</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Population-Dataset">Zurich Population Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903">Stadt Zürich</a></td>
            <td>Population by neighbourhood, origin, sex and age, since 1993.</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Income-Data">Zurich Income Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003">Stadt Zürich</a></td>
            <td>Median income of taxable individuals by year, tax rate and urban district, since 1999</td>
        </tr>
        <tr>
            <td><a href="#Zurich-Household-Dataset">Zurich Household Data</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv">Link</a></td>
            <td><a href="https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806">Stadt Zürich</a></td>
            <td>Private households by household size and urban district, since 2013.</td>
        </tr>
    </tbody>
</table>

<p>These datasets collectively enable a comprehensive analysis of dog ownership trends in Zurich.</p>


### Data Preprocessing

##### Column Name Transformation
To enhance readability and ensure consistency across datasets, original column names were translated from German to English and standardized to snake case using our `sanitize_df_column_names` helper function. This transformation facilitates a cleaner, more uniform `pd.DataFrame` structure for analysis.


#### Zurich Statistical Districts Geospatial Data

This first geodataset comes as a compressed file containing 3 geojson files.

1. `z_gdf_0`: point geometry data at the ideal position for placing a number label on the polygon map.

2. `z_gdf_1`: polygon geometry data specifically for visual representation in cartography i.e.maps.

3. `z_gdf_2`: polygon geometry data recommended for use for accurate geometry calculations, like spatial joins or area calculations.

Together these three files provide excellent geodedic information on the geographical region of Zürich for our analysis.

In [ ]:
# save the url of the website
zurich_districts_url = "https://www.zuerich.com/en/visit/about-zurich/zurichs-districts"

zurich_desc = hf.get_zurich_description(zurich_districts_url)
# Create a 'link' column in the description DataFrame with links to each district's details.
zurich_desc["link"] = zurich_desc["district"].apply(
    lambda x: f"{zurich_districts_url}#s-{x}"
)
# Display the Zurich districts description DataFrame.
zurich_desc.info()

# Define the URL for the Zurich Statistical Quarters geospatial data ZIP file.
zip_gdf_url = "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_statistical_quarters.zip"

# Load the geospatial data into Zurich Geo DataFrames.Would you prefer if we do
zurich_geo_dicts = hf.get_gdf_from_zip_url(zip_gdf_url)

# Rename keys in the Zurich Geo DataFrames with a prefix.
z_gdf = hf.rename_keys(zurich_geo_dicts, prefix="z_gdf_")

# Display the information and a sample of data from each GeoDataFrame in the z_gdf dictionary
for key in z_gdf.keys():
    print(f"Information for {key}:")
    z_gdf[key].info()
    print(f"Sample data from {key}:")
    display(z_gdf[key].sample(3))

#### Zurich Dogs Dataset

In [ ]:
zurich_dog_data_link = "https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv"
zurich_dog_data_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_dogs.csv"
)
zurich_dog_data = pd.read_csv(zurich_dog_data_link)
zurich_dog_data.info()

zurich_dog_data = hf.sanitize_df_column_names(zurich_dog_data)
zurich_dog_data.info()
zurich_dog_data.sample(3)

#### Zurich Population Dataset



In [ ]:
# zurich_pop_link = "https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv"
zurich_pop_link = "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_pop.csv"
zurich_pop_data = pd.read_csv(zurich_pop_link)
zurich_pop_data = hf.sanitize_df_column_names(zurich_pop_data)
zurich_pop_data.info()
zurich_pop_data.sample().T

#### Zurich Income Dataset
These data contain quantile values of the taxable income of natural persons who are primarily taxable in the city of Zurich. Tax income are in thousand francs (integer).

In [ ]:
# zurich_income_link = "https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv"
zurich_income_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_income.csv"
)
zurich_income_data = pd.read_csv(zurich_income_link)
zurich_income_data.info()

# Clean column names, display info and sample
zurich_income_data = hf.sanitize_df_column_names(zurich_income_data)


zurich_income_data.info()
zurich_income_data.sample(5)

#### Zurich Household data 

In [1]:
# zurich_household_data_link = "https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv"
zurich_household_data_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_household.csv"
)
zurich_household_data = pd.read_csv(zurich_household_data_link)
zurich_household_data.info()

zurich_household_data = hf.sanitize_df_column_names(zurich_household_data)
zurich_household_data.info()
zurich_household_data.head(5)

NameError: name 'pd' is not defined

# Inspect the Dataset
Use functions like head(), info(), and describe() to inspect various aspects of the dataset.


The Zurich geospatial data has been transformed for improved clarity and usability. Key changes include:
- **Column Renaming:** `qname` to `neighborhood`, `qnr` to `sub_district`, `knr` to `district`.
- **Data Type Adjustments:** `sub_district` formatted as a string with leading zeros.
- **Column Selection:** Refined `neighborhood_gdf` with key columns: `neighborhood`, `sub_district`, `district`, `geometry`.
- **Coordinate Reference System (CRS):** Set to WGS 84 (EPSG:4326) for accurate geodetic latitude and longitude.

The result is a streamlined `neighborhood_gdf`, offering a clear and structured representation of Zurich's districts for further spatial analysis.


In [ ]:
zurich_map_gdf = z_gdf["z_gdf_1"]

zurich_map_gdf.rename(
    columns={"qname": "neighborhood", "qnr": "sub_district", "knr": "district"},
    inplace=True,
)
# Format the sub_district column to have 3 digits
zurich_map_gdf["sub_district"] = zurich_map_gdf["sub_district"].astype(str).str.zfill(3)

# Create the refined geodataframe
neighborhood_gdf = zurich_map_gdf[
    ["neighborhood", "sub_district", "district", "geometry"]
].copy()

# Display geodataframe information and CRS
neighborhood_gdf.info()
display(neighborhood_gdf.crs)

# Display a sample entry from the transformed geodataframe
neighborhood_gdf.sample().T

In [ ]:
# Load the geospatial data for calculation
zurich_calc_gdf = z_gdf["z_gdf_2"]

# Calculate area in square meters and add as a new column
zurich_calc_gdf["area_km2"] = zurich_calc_gdf.to_crs(ccrs.GOOGLE_MERCATOR).area / 1e6

# Rename the column for consistency with the main geodataframe
zurich_calc_gdf = zurich_calc_gdf.rename(columns={"qname": "neighborhood"})

# Merge calculated features with the main geodataframe (neighborhood_gdf)
area_gdf = neighborhood_gdf.merge(
    zurich_calc_gdf[["neighborhood", "area_km2"]], on="neighborhood"
)

# Display a snapshot of the merged geodataframe
print(area_gdf[["neighborhood", "area_km2"]].head())

# Clean the Dataset
Handle missing values, outliers, and incorrect data types to clean the dataset.

In [ ]:
# Check for missing values in the dataframe
missing_values = df.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Fill missing values with appropriate method
# Here, we are using forward fill method. You can use 'bfill', mean, median or any other method based on your data
df.fillna(method='ffill', inplace=True)

# Check for outliers in the dataframe
# Here, we are using Z-score method to detect outliers. You can use IQR, box plots or any other method based on your data
from scipy import stats
z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
outliers = (z_scores > 3).any(axis=1)  # Here, 3 is the threshold, you can adjust it based on your data
print("Number of outliers in the dataframe:", outliers.sum())

# Remove outliers from the dataframe
df = df[~outliers]

# Check for incorrect data types in the dataframe
print("Data types of each column:\n", df.dtypes)

# Convert incorrect data types to correct data types
# Here, we are assuming 'column1' should be of type 'float'. Replace 'column1' with your column name
# df['column1'] = df['column1'].astype('float')

# Now, the dataframe is cleaned and ready for further analysis
df.head()

# Transform the Dataset
Perform necessary transformations on the data, such as normalization, scaling, or encoding categorical variables.

In [ ]:
# Importing the required libraries for data transformation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

# Normalizing numerical columns
# Here, we are assuming 'numerical_column1' and 'numerical_column2' are numerical columns. Replace these with your column names
numerical_columns = ['numerical_column1', 'numerical_column2']  # Add more column names if needed
scaler = StandardScaler()  # You can use MinMaxScaler or any other scaler based on your data
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Encoding categorical columns
# Here, we are assuming 'categorical_column1' and 'categorical_column2' are categorical columns. Replace these with your column names
categorical_columns = ['categorical_column1', 'categorical_column2']  # Add more column names if needed
encoder = LabelEncoder()
for column in categorical_columns:
    df[column] = encoder.fit_transform(df[column])

# Display the transformed dataframe
df.head()

# Visualize the Cleaned and Transformed Data
Use matplotlib or seaborn to visualize the cleaned and transformed data to identify patterns, trends, and outliers.

In [ ]:
# Importing seaborn for advanced data visualization
import seaborn as sns

# Visualizing the distribution of numerical columns
for column in numerical_columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.show()

# Visualizing the count of categorical columns
for column in categorical_columns:
    plt.figure(figsize=(10, 6))
    sns.countplot(x=df[column])
    plt.title(f'Count of {column}')
    plt.show()

# Visualizing the correlation between numerical columns
plt.figure(figsize=(10, 6))
sns.heatmap(df[numerical_columns].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation between numerical columns')
plt.show()

# Visualizing the relationship between numerical columns using pairplot
sns.pairplot(df[numerical_columns])
plt.show()

# Visualizing the relationship between categorical and numerical columns using boxplot
for numerical_column in numerical_columns:
    for categorical_column in categorical_columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=df[categorical_column], y=df[numerical_column])
        plt.title(f'Relationship between {categorical_column} and {numerical_column}')
        plt.show()